In [ ]:
from IPython.core.display import HTML
with open ("../style.css") as file:
    css = file.read()
HTML(css)

# A Grammar for Propositional Logic

This file shows how a simple symbolic calculator can be implemented using `Ply`.  The grammar for the language implemented by this parser is as follows:
$$
\begin{array}{lcl}
  \texttt{formula} & \rightarrow & \;\texttt{formula}\; \texttt{'↔'} \; \texttt{formula}                \\
                   & \mid        & \;\texttt{formula}\; \texttt{'→'} \; \texttt{formula}                \\
                   & \mid        & \;\texttt{formula}\; \texttt{'∨'} \; \texttt{formula}                \\
                   & \mid        & \;\texttt{formula}\; \texttt{'∧'} \; \texttt{formula}                \\
                   & \mid        & \;\texttt{'¬'} \;\texttt{formula}                                    \\
                   & \mid        & \;\texttt{'('} \; \texttt{formula} \;\texttt{')'}                    \\
                   & \mid        & \;\texttt{'⊤'}                                                       \\
                   & \mid        & \;\texttt{'⊥'}                                                       \\
                   & \mid        & \;\texttt{IDENTIFIER}                        
\end{array}
$$

## Specification of the Scanner

$\leftrightarrow$, $\rightarrow$, $\wedge$, $\vee$, $\neg$, $\top$, $\bot$

In [ ]:
import ply.lex as lex

There are only five tokens that need to be defined via regular expressions.  The other tokens consist only of a single character and are therefore 
defined as literals.

In [ ]:
tokens = [ 'IDENTIFIER' ]

The token `IDENTIFIER` specifies the name of a *variable*.  It may contain the angle brackets `<` and `>`.

In [ ]:
def t_IDENTIFIER(t):
    r'[a-zA-Z][a-zA-Z0-9_<>]*'
    return t

`literals` is a list operator symbols that consist of a single character.

In [ ]:
literals = ['↔', '→', '∧', '∨', '¬', '⊤', '⊥', '(', ')']

Blanks and tabulators are ignored.

In [ ]:
t_ignore  = ' \t'

Newlines are counted in order to give precise error messages.  Otherwise they are ignored.

In [ ]:
def t_newline(t):
    r'\n+'
    t.lexer.lineno += t.value.count('\n')

Unkown characters are reported as lexical errors.

In [ ]:
def t_error(t):
    print(f"Illegal character '{t.value[0]}' at character number {t.lexer.lexpos} in line {t.lexer.lineno}.")
    t.lexer.skip(1)

In [ ]:
__file__ = 'main'

We generate the lexer.

In [ ]:
lexer = lex.lex()

## Specification of the Parser

In [ ]:
import ply.yacc as yacc

The grammar rules for `formula` are:
```
 formula : formula '↔' formula
         | formula '→' formula
         | formula '∧' formula
         | formula '∨' formula
         | '¬' formula
         | '⊤'
         | '⊥'
         | '(' formula ')'
         | IDENTIFIER
         ;
```

In [ ]:
def p_formula_equivalence(p):
    "formula : formula '↔' formula"
    p[0] = ('↔', p[1], p[3])

def p_formula_implication(p):
    "formula : formula '→' formula"
    p[0] = ('→', p[1], p[3])

def p_formula_or(p):
    "formula : formula '∨' formula"
    p[0] = ('∨', p[1], p[3])
    
def p_formula_and(p):
    "formula : formula '∧' formula"
    p[0] = ('∧', p[1], p[3])
    
def p_formula_not(p):
    "formula : '¬' formula"
    p[0] = ('¬', p[2])
    
def p_formula_verum(p):
    "formula : '⊤'"
    p[0] = ('⊤',)
    
def p_formula_falsum(p):
    "formula : '⊥'"
    p[0] = ('⊥',)
    
def p_formula_paren(p):
    "formula : '(' formula ')'"
    p[0] = p[2]
    
def p_formula_identifier(p):
    "formula : IDENTIFIER"
    p[0] = p[1]

In [ ]:
precedence = ( ('nonassoc', '↔'),
               ('right',    '→'),
               ('left',     '∨'),
               ('left',     '∧'),
               ('right',    '¬')
             )

The method `p_error` is called if a syntax error occurs.  The argument `p` is the token that could not be read.  If `p` is `None` then there is a syntax error at the end of input.

In [ ]:
def p_error(p):
    if p:
        print(f"Syntax error at character number {p.lexer.lexpos} at token '{p.value}' in line {p.lexer.lineno}.")
    else:
        print('Syntax error at end of input.')

Setting the optional argument `write_tables` to `False` <B style="color:red">is required</B> to prevent an *obscure bug* where the parser generator tries to read an empty parse table.
We set `debug` to `True` so that the parse tables are dumped into the file `parser.out`.

In [ ]:
parser = yacc.yacc(write_tables=False, debug=True)

Let's look at the action table that is generated.

In [ ]:
!cat parser.out

In [ ]:
%run ../AST2Dot.ipynb

The method `test(s)` takes a string `s` that is supposed to be a `stmnt`.
This statement is then executed.

In [ ]:
def test(s):
    t = yacc.parse(s)
    d = tuple2dot(t)
    display(d)
    return t

$\leftrightarrow$, $\rightarrow$, $\wedge$, $\vee$, $\neg$, $\top$, $\bot$

In [ ]:
test('(p ↔ q) ∨ (q ↔ p) → (¬q ↔ ¬p) ∧ (q → ⊥ ↔ p → ⊥)')

In [ ]:
test('p ∧ ⊤')

In [ ]:
test('q ∨ ⊥')

In [ ]:
test('¬p ∧ q')

In [ ]:
test('p ∧ q → r')

Implication is right-associative.

In [ ]:
test('p → q → r')

In [ ]:
test('¬(p ∨ q) ↔ ¬p ∧ ¬q')

Negation is right associative.

In [ ]:
test('¬¬p ↔ p')

In [ ]:
test('(p → q) ∧ (q → r) → p → r')

`∧` binds stronger than `∨`.

In [ ]:
test('p ∨ q ∧ r')

In [ ]:
test('v ↔ u → p ∨ q ∧ ¬ ⊥')